# Task 1:

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

# Load the pre-trained model from TensorFlow Hub
num_classes = 5
module_url = "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/5"
model = tf.keras.Sequential([
    hub.KerasLayer(module_url, trainable=False),
    layers.Dense(num_classes, activation='softmax')  # num_classes is the number of classes in your dataset
])

# Load and preprocess the dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    'C:/Users/Ali/Downloads/labs/lab1/fdataset/train',
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    'C:/Users/Ali/Downloads/labs/lab1/fdataset/val',
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

# Configure the dataset for performance
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)

# Compile the model
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# Train the model
epochs = 10 # Adjust the number of epochs as needed
history = model.fit(train_dataset, epochs=epochs, validation_data=validation_dataset)

# Evaluate the model
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    'C:/Users/Ali/Downloads/labs/lab1/fdataset/train',
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

model.evaluate(test_dataset)



ModuleNotFoundError: No module named 'tensorflow_hub'